<a href="https://colab.research.google.com/github/mbalbi/ciencia_de_datos/blob/main/tps/trabajo_practico_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Trabajo Práctico 4: Modelos de regresión

**Objetivos:** Este trabajo tiene como objetivos familiarizar al alumno con los métodos numéricos para el ajuste de ecuaciones de regresión con un framework Bayesiano, así como entender el efecto de la incertidumbre de los parametros estimados en las predicciones y el análisis de datos.

**Librerías:** para realizar este trabajo serán necesarias las siguientes librerías (Si requiere usar otras no incluidas aquí, no dude en importarlas).
- Numpy: matemática básica y definición de matrices y vectores
- Scipy.stats: repositorio de distribuciones de probabilidad y funciones de algebra
- Pandas: manejo de datasets
- Seaborn: visualización de datos
- pymc: inferencia bayesiana
- arviz:

In [ ]:
import numpy as np
import scipy.stats as st
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import io
# !pip install pymc
import pymc as pm
import arviz as az

## Ejercicio 1: Inferencia de la distribución de probabilidad de distintos tipos de hormigón.

Se analizarán los resultados de los ensayos de distintos hormigones a 28 días, cuya composición se encuentra indicada en el dataset.


###Curado del Dataset

Primero, cargue el dataset **concrete_data_28days.csv**, en el que encontrará todas las variables necesarias para realizar la regresión:

In [ ]:
#Abrimos un archivo local, es decir, que ha sido descargado en su computadora
from google.colab import files
uploaded = files.upload()

In [ ]:
#Se crea un DataFrame
df_hormigon = pd.read_csv(io.BytesIO(uploaded['concrete_data_28days.csv']))
df_hormigon.head()

**Item**: Lo primero que debe hacer es crear una nueva columna, llamada  `strength_cat` equivalente a las variables ordinales categóricas de la columna `type` según:
- baja: 1
- media: 2
- alta: 3

In [ ]:
# CODIGO DEL ALUMNO ############################################################



################################################################################

df_hormigon.head()

###Análisis Exploratorio de Datos

A continuación, se grafica la matriz de correlación para identificar aquellas variables que podrían ser interesantes utilizar para modelar.

In [ ]:
# Gráfico de calor para la matriz de correlación
sns.heatmap(df_hormigon.corr(), cmap='BuPu', fmt='.2f', annot=True, linewidths=.6)

A continuación, se graficarán distintas relaciones.

**Item**: ¿Observa alguna tendencia de cómo la cantidad de los materiales analizados afectan a la resistencia del hormigón?

**Su respuesta**

In [ ]:
sns.scatterplot(y="concrete_compressive_strength", x="cement", hue="water", data=df_hormigon,sizes=(50, 300))
plt.legend(bbox_to_anchor=(1.02, 1), loc='upper left', borderaxespad=0, title='water').set(title='agua')

In [ ]:
sns.scatterplot(y="concrete_compressive_strength", x="cement", hue="water", size="superplasticizer", data=df_hormigon,sizes=(50, 300)).set(title='Resistencia del hormigón')
plt.legend(bbox_to_anchor=(1.02, 1), loc='upper left', borderaxespad=0)

In [ ]:
sns.scatterplot(y="concrete_compressive_strength", x="cement", hue="water", size="fly_ash", data=df_hormigon,sizes=(50, 300)).set(title='Resistencia del hormigón')
plt.legend(bbox_to_anchor=(1.02, 1), loc='upper left', borderaxespad=0)

###Construcción de Modelos

Siendo $strength$_$cat$ una variable categórica ordinal, se proponen distintos modelos para luego evaluar con cuál de ellos quedarse.

In [ ]:
# Preparación de la data input
y = df_hormigon['strength_cat'].to_numpy() 
x = df_hormigon[['cement', 'water', 'fly_ash', 'superplasticizer']]

Nk = 3 # Número de categorias

####Modelo 1

Para empezar, el Modelo 1 será el más sencillo y tendrá a $cement$ como única variable. Le recomendamos ver detenidamente la sintaxis escrita por nosotros para que luego la repita en el siguiente modelo.

In [ ]:
# Estandarización de las variables
x_cement_std = x['cement']
x_cement_std = (x_cement_std-x_cement_std.mean())/x_cement_std.std()

In [ ]:
with pm.Model() as model_1:

    X_cement = pm.MutableData( "x_cement", x_cement_std)

    # Distribuciones a priori
    kappa = pm.Normal( "kappa", mu=0.0, sigma=2,
                       transform=pm.distributions.transforms.ordered,
                       shape=Nk, initval=np.arange(Nk) - 2 )
        
    beta_cement = pm.Normal("beta_cement", 0, 1)

    # Probabilidades
    phi = pm.Deterministic( "phi", beta_cement*X_cement)
    
    # Verosimilitud
    Y = pm.OrderedLogistic("y", eta=phi, cutpoints=kappa, observed=y-1)

pm.model_to_graphviz(model_1)

In [ ]:
with model_1:
  trace_1 = pm.sample( 3000, tune=2000, cores=2, target_accept=0.9 )

**Item**: Selecciona un intervalo para generar simulaciones posteriores predictivas para graficar xtest:

In [ ]:
# CODIGO DEL ALUMNO ############################################################
xtest =

################################################################################

with model_1:
    pm.set_data({"x_cement":xtest})
    ppc_1 = pm.sample_posterior_predictive( trace_1, var_names=['y','phi','y_probs'] )

**Item**: Realiza el gráfico de distribuciones posteriores $P[DS>=dsi]$ para cada categoría, en función de la variable $cement$:

In [ ]:
from matplotlib.ticker import MultipleLocator

# CODIGO DEL ALUMNO ############################################################



################################################################################

#### Modelo 2

**Item**: De forma análoga al modelo 1, propone una nueva regresión con las variables analizadas en el EDA. 

In [ ]:
# Estandarización de las variables
# CODIGO DEL ALUMNO ############################################################



################################################################################

In [ ]:
# Modelo: model_2
# CODIGO DEL ALUMNO ############################################################



################################################################################

pm.model_to_graphviz(model_2)

### Selección del Modelo

**Item**: A partir de la estimación del Leave-One-Out Cross-Validation (PSIS-LOO CV), haga un ranking de los modelos elaborados y seleccione el modelo que crea más adecuado

In [ ]:
# Comparamos los modelos polinómicos mediane PSIS-LOOCV
models_list = {'1':trace_1,'2':trace_2}

# CODIGO DEL ALUMNO ============================================================


# ==============================================================================

az_comp_loo

**Item**: Muestre en forma gráfica los resultados

In [ ]:
# CODIGO DEL ALUMNO ============================================================


# ==============================================================================

**Item**: Indique qué modelo seleccionó. Viendo el ordenamiento de los modelos, ¿incorporar más variables siempre mejora el modelo?

**Su Respuesta**:

## Ejercicio 2: Regresión con procesos gaussianos 

El objetivo de este segundo ejercicio es que explore los distintos modelos de regresión para su dataset seleccionado. Para ello:
- Construya **un** modelo con procesos graussianos
